<a href="https://colab.research.google.com/github/ViralTexts/viral-texts-classification/blob/workshop/RecipeClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
install.packages("tidytext")
library(tidyverse)
library(tidytext)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘SnowballC’, ‘hunspell’, ‘janeaustenr’, ‘tokenizers’




_Viral Texts_ RA Avery Blankenship wrote more about why she assembled this corpus of nineteenth-century receipes in this blog post, which also links to the text files if you want to read some of them: <https://viraltexts.org/2021/02/24/c19-recipes/>

In [11]:
texts <- read_csv("https://github.com/ViralTexts/viral-texts-classification/raw/workshop/hand-tagged/recipes.csv?raw=true")

head(texts)

Rows: 1034 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): doc_id, text, genre

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


doc_id,text,genre
<chr>,<chr>,<chr>
29982_anewmayonnaise.txt,"A NEW MAYONNAISE (Soyer's).--Put a quarter of a pint of stiff veal jelly (that has been nicely flavored with vegetables) on ice in a bowl, whisking it till it is a white froth; then add half a pint of salad oil and six spoonfuls of tarragon vinegar, _by degrees_, first oil, then vinegar, continually whisking till it forms a white, smooth, sauce-like cream; season with half a teaspoonful of salt, a quarter ditto of white pepper, and a very little sugar, whisk it a little more and it is ready. It should be dressed pyramidically over the article it is served with. The advantage of this sauce is that (although more delicate than any other) you may dress it to any height you like, and it will remain so any length of time; if the temperature is cool, it will remain hours without appearing greasy or melting. It is absolutely necessary, however, that it should be prepared on ice.",recipe
29982_anicelittlebreakfastdish.txt,"A NICE LITTLE BREAKFAST DISH IS made thus: Cut two long slices of cold meat and three of bread, buttered thickly, about the same shape and size; season the meat with pepper, salt, and a little finely chopped parsley; or, if it is veal, a little chopped ham; then lay one slice of bread between two of meat, and have the other two slices outside; fasten together with short wooden skewers. If you have a quick oven, put it in; and take care to baste with butter thoroughly, that the bread may be all over crisp and brown. If you can't depend on your oven, fry it in very hot fat as you would crullers; garnish with sprigs of parsley, and serve very hot.",recipe
29982_anothervealandhampie.txt,"ANOTHER VEAL AND HAM PIE, more usual, and probably the ""weal and hammer"" that ""mellered the organ"" of Silas Wegg, was manufactured by Mrs. Boffin from this recipe; it is as follows: Take the thick part of breast of veal, removing all the bones, which put on for gravy, stewing them long and slowly; put a layer of veal, pepper and salt, then a thin sprinkling of ham; if boiled, cut in slices; if raw, cut a slice in dice, which scald before using, then more veal and again ham. If force-meat balls are liked, make some force-meat as for Windsor pie, using if you prefer it chopped hard-boiled eggs in place of chopped meat, and binding into a paste with a raw egg; then make into balls, which drop into the crevices of the pie; boil two or three eggs quite hard, cut each in four and lay them round the sides and over the top, pour in about a gill of gravy, and cover the same as the Windsor pie. In either of these pies the force-meat may be left out, a sweetbread cut up, or mushrooms put in. A chicken pie to eat cold is very fine made in this way.",recipe
29982_baba.txt,"BABA.--One pound of flour; take one quarter of it, and make a sponge with half an ounce of compressed yeast and a little warm water, set it to rise, make a hole in the rest of the flour, add to it ten ounces of butter, three eggs, and a dessert-spoonful of sugar, a little salt, unless your butter salts it enough, which is generally the case. Beat all together well, then add five more eggs, one at a time, that is to say, add one egg and beat well, then another and beat again, and so on until the five are used. When the paste leaves the bowl it is beaten enough, but not before; then add the sponge to it, and a large half ounce of citron chopped, the same of currants, and an ounce and a half of sultana raisins, seedless. Let it rise to twice its size, then bake it in an oven of dark yellow paper heat; the small round babas are an innovation of the pastry-cook to enable him to sell them uncut. But the baba proper should be baked in a large, deep, upright tin, such as a large charlotte russe mold, when they keep for several days fresh, and if they get stale, make delicious fritters, soaked in sherry and dipped in frying batter. In some cases, however, it may be preferred to make them as usually seen at French pastry cook

In [17]:
tidy_texts <- texts %>%
  unnest_tokens(word, text) %>%
  group_by(word) %>%
  anti_join(stop_words) %>%
  ungroup()

head(tidy_texts)

Joining, by = "word"


doc_id,genre,word
<chr>,<chr>,<chr>
29982_anewmayonnaise.txt,recipe,mayonnaise
29982_anewmayonnaise.txt,recipe,soyer's
29982_anewmayonnaise.txt,recipe,quarter
29982_anewmayonnaise.txt,recipe,pint
29982_anewmayonnaise.txt,recipe,stiff
29982_anewmayonnaise.txt,recipe,veal


In [ ]:
test_data <- sample_n(texts, 100, replace = FALSE) %>%
  mutate(genre = "unknown")
train_data <- anti_join(texts, test_data) %>%
  mutate(genre = "recipe")

In [ ]:
# head(test_data)
# head(train_data)

In [38]:
sparse_words <- tidy_texts %>%
  count(doc_id, word) %>%
  cast_sparse(doc_id, word, n)
class(sparse_words)



colnames(sparse_words[,20:39])
sparse_words[1:20, 20:39]


[1] "dgCMatrix"
attr(,"package")
[1] "Matrix"

[1] "half"          "height"        "hours"         "ice"          
 [5] "jelly"         "length"        "mayonnaise"    "melting"      
 [9] "nicely"        "oil"           "pepper"        "pint"         
[13] "prepared"      "pyramidically" "quarter"       "ready"        
[17] "remain"        "salad"         "salt"          "sauce"

   [[ suppressing 20 column names ‘half’, ‘height’, ‘hours’ ... ]]



20 x 20 sparse Matrix of class "dgCMatrix"
                                                                               
29982_anewmayonnaise.txt                2 1 1 2 1 1 1 1 1 2 1 2 1 1 2 1 2 1 1 2
29982_anicelittlebreakfastdish.txt      . . . . . . . . . . 1 . . . . . . . 1 .
29982_anothervealandhampie.txt          . . . . . . . . . . 1 . . . . . . . 1 .
29982_baba.txt                          6 . . . . . . . . . . 2 . . 1 . . . 1 .
29982_beefaugratin.txt                  . . . . . . . . . . . . . . . . . . . .
29982_beefolives.txt                    2 . . . . . . . . . . . . . . . . . . .
29982_biscuitglacealacharlesdickens.txt 3 . . 3 . . . . . . . 3 . . . 1 . . . .
29982_biscuitglacealathackeray.txt      2 . . . . . . . . . . 3 . . . . . . . .
29982_boeufalajardiniere.txt            . . . . . . . . . . 1 1 . . . . . . 1 .
29982_boucheesdesdames.txt              3 . . 2 7 . . . . . . . 1 . . . . . . .
29982_brioche.txt                       . . . . . . . . . . . . . . . 1 . . .

In [39]:
texts_joined <- rbind(test_data, train_data)

In [41]:
install.packages("glmnet")
library(glmnet)
library(doMC)
library(broom)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘iterators’, ‘foreach’, ‘shape’, ‘RcppEigen’


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loaded glmnet 4.1-3



ERROR: ignored

In [ ]:
registerDoMC(cores = 8)
is_recipe <- texts_joined$genre == "recipe"
model <- cv.glmnet(sparse_words, is_recipe,
                   family = "binomial",
                   parallel = TRUE, keep = TRUE)

coefs <- model$glmnet.fit %>%
  tidy() %>%
  filter(lambda == model$lambda.1se)